In [1]:
# Use Huggingface ViLT

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import os
import sys
import json
import argparse
#from torchvision import transforms
from transformers import ViltProcessor, ViltForQuestionAnswering, ViltConfig
from PIL import Image
import pandas as pd
import requests

In [2]:
# prepare image and text
url = "https://prod-images-static.radiopaedia.org/images/17484000/e9bcb98e83a7c84e5f2a1cf890dbf6_big_gallery.jpeg"
image = Image.open(requests.get(url, stream=True).raw).convert("RGB")

image.show()

In [3]:
# Define the user question
question = "is the lung healthy?"

In [4]:
from transformers import pipeline
from PIL import Image
import requests

# Initialize the pipeline for image-to-text using the BLIP image captioning model
pipe = pipeline("image-to-text", model="Salesforce/blip-image-captioning-large", device='cuda')

# Open an image (replace 'your_image.jpg' with the path to your image)
#img_url = 'https://prod-images-static.radiopaedia.org/images/4170261/c5d7c3ed6c7fe53e59c2dd902e44b9_big_gallery.jpg'
#image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')

# Generate caption for the image
result = pipe(image)

# Display the result (caption)
print(result[0]['generated_text'])


/home/mehdi/Desktop/aut/pkgs/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/home/mehdi/Desktop/aut/pkgs/lib/python3.10/site-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


a black and white photo of a chest with a large lung


In [5]:
Photo_Context = result[0]['generated_text']

In [9]:
# Initialize the pipeline for text generation using the Medical-Llama3-8B model
#pipe = pipeline("text-generation", model="ruslanmv/Medical-Llama3-8B", device='cuda')

from transformers import BitsAndBytesConfig
quant_config = BitsAndBytesConfig(load_in_4bit=True)
pipe = pipeline("text-generation", model="ruslanmv/Medical-Llama3-8B", device='auto', quantization_config=quant_config)

# Generate text
prompt = f"""
Photo Context: {Photo_Context}
User Question: {question}
Task: Create a list of question in Python list syntax for the information that you need to answer the question
"""
print(prompt)

result = pipe(prompt, max_length=200, num_return_sequences=1)

# Display the result
print(result[0]['generated_text'])

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [13]:
question_list_str = result[0]['generated_text']

# Convert the generated text to a list
question_list =eval(question_list_str)

print(question_list)

AttributeError: 'str' object has no attribute 'to_list'

In [2]:
# Load the model VILT
config = ViltConfig.from_pretrained("vilt-Med_RAD-finetuned-vqa-KHC-4-with_valid_labels")
model = ViltForQuestionAnswering.from_pretrained("vilt-Med_RAD-finetuned-vqa-KHC-Model-merged_datasets-config-extend-with_valid",
                                                 id2label=config.id2label,
                                                 label2id=config.label2id)
processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-mlm")

In [ ]:
# run all the questions in the list and get the answer for each question from VILT

answers = []

for q in question_list:
    # prepare inputs
    encoding = processor(image, q, return_tensors="pt")

    # forward pass
    outputs = model(**encoding)
    logits = outputs.logits
    idx = logits.argmax(-1).item()
    print("Predicted answer:", model.config.id2label[idx])
    answers.append(model.config.id2label[idx])

In [ ]:
# input the answer list back to the text generation model to generate the final answer

# Initialize the pipeline for text generation using the Medical-Llama3-8B model
pipe = pipeline("text-generation", model="ruslanmv/Medical-Llama3-8B")

# Generate text
prompt = f"""
Photo Context: {Photo_Context}
User Question: {question}
Questions and Answers: {list(zip(question_list, answers))}
Task: Generate the final answer
"""
print(prompt)

result = pipe(prompt, max_length=200, num_return_sequences=1)

# Display the result
print(result[0]['generated_text'])